In [ ]:
# テキストコーパスをチャンクに分割
with open("./RAGdata/query.txt", encoding = "utf-8") as f:
    text = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
)

texts = text_splitter.split_text(text)

# LangChainで利用する埋め込みモデルの設定
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={
        'device': 'cuda:0'
    }
)

from langchain.vectorstores import FAISS
db = FAISS.from_texts(texts, embeddings)
# データベースを保存
db.save_local("./RAGdata/query.db")

In [23]:
# LangChainで利用する埋め込みモデルの設定
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={
        'device': 'cuda:0'
    }
)

# データベースの読み込み
from langchain.vectorstores import FAISS
db = FAISS.load_local(
    "./RAGdata/query.db",
    embeddings,
    allow_dangerous_deserialization = True
)

# データベースからの情報検索のためのリトリーバーを作成
retriever = db.as_retriever(
    search_kwargs = {'k': 3},
)

# プロンプトの準備
template = """
ユーザー: 以下のテキストから「ユーザーが実際に尋ねている質問文」を1つだけ抽出してください。ユーザーの質問文を1文だけ。余計な説明は出力しない。
{context}

{question}

システム:"""

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template = template,
    input_variables = ["context", "question"],
    template_format = "f-string",
)

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

modelID = "line-corporation/japanese-large-lm-3.6b-instruction-sft"

tokenizer = AutoTokenizer.from_pretrained(
    modelID,
    legacy = False,
    use_fast = False
)

model = AutoModelForCausalLM.from_pretrained(
    modelID,
    torch_dtype = torch.float16,
    device_map = "auto",
    low_cpu_mem_usage = True
).eval()

pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 64,
    do_sample = True,
    temperature = 0.7,
    repetition_penalty = 2.0,
)

from langchain.chains import RetrievalQA
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

qa = RetrievalQA.from_chain_type(
    llm = HuggingFacePipeline(pipeline = pipe),
    retriever = retriever,
    chain_type = "stuff",
    return_source_documents = True,
    chain_type_kwargs = {
        "prompt": prompt,
    },
    verbose = True,
)

q = "質問はありましたか？"
ans = qa.invoke(q)
print(ans['result'])

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0




> Entering new RetrievalQA chain...

> Finished chain.

ユーザー: 以下のテキストから「ユーザーが実際に尋ねている質問文」を1つだけ抽出してください。ユーザーの質問文を1文だけ。余計な説明は出力しない。
教えてください。何か毎日続けている習慣はありますか？休日はどのように過ごすことが多いですか？あなたの得意料理は何ですか？健康のために何か気をつけていることはありますか？普段、音楽は聴きますか？どんなジ

て何を聞きたいですか？もし動物と話せるとしたら、どの動物と何を話したいですか？もし自分が国のリーダーになったら、最初に取り組むことは何ですか？もし人生をゲームに例えるなら、今はどのステージにいると思い

詳しいと自信を持って言える分野は何ですか？今、一番学びたいと思っていることは何ですか？学校の勉強で、一番好きだった科目と嫌いだった科目は何ですか？どうやって新しい情報を得ることが多いですか？人に何かを

質問はありましたか？

システム: ウェブ
本文: 第2回となる今回は,これまでの議論を踏まえて作成したガイドラインの概要を紹介します.この概要には3つの目的があり(図参照),それぞれの目的を達成するために設定した内容は以下の通りです;-) <その壱>ではまずはじめに現状の問題点をいくつか紹介し<其の二 >へ移行します (例によって箇条書き


In [5]:
a = db.similarity_search("質問はありましたか？", k=3)
print(len(a))
print(type(a[0]))
print(a[0].page_content)

3
<class 'langchain_core.documents.base.Document'>
教えてください。何か毎日続けている習慣はありますか？休日はどのように過ごすことが多いですか？あなたの得意料理は何ですか？健康のために何か気をつけていることはありますか？普段、音楽は聴きますか？どんなジ


In [9]:
# データベースの構築
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name = "intfloat/multilingual-e5-large",
    model_kwargs = {'device':'cuda:0'},
#    encode_kwargs = {'normalize_embeddings': False}
)

from langchain_community.vectorstores import FAISS

# 保存してあるデータベースを読み込む
db = FAISS.load_local('./RAGdata/query.db',embeddings, allow_dangerous_deserialization=True)


# 検索機の構築
# retriever = db.as_retriever()
retriever = db.as_retriever(search_kwargs={'k':2})


# モデルの準備
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "line-corporation/japanese-large-lm-3.6b-instruction-sft"

tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
#    device_map="cuda:0",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).eval()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.01,
    repetition_penalty=2.0,
)

# プロンプトの準備
template = """
ユーザー: 以下のテキストから「ユーザーが実際に尋ねている質問文」を1つだけ抽出してください。
{context}

{question}

システム:"""

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)

# RetrievalQAのインスタンス作成
from langchain.chains import RetrievalQA
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

qa = RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(pipeline=pipe),
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

# 実行
q = "ひとつの質問を抽出してください。"
ans = qa.invoke(q)
print(ans['result'])

print("--------------------------")

import re
pattern = re.compile(r'システム:(.*)',re.DOTALL)
match = pattern.search(ans['result'])
ans0 = match.group(1)
print(ans0)

# docs = ans['source_documents']
# for d in docs:
#     print()
#     print(d.page_content[:100])

Device set to use cuda:0




> Entering new RetrievalQA chain...

> Finished chain.

ユーザー: 以下のテキストから「ユーザーが実際に尋ねている質問文」を1つだけ抽出してください。
あなたはどうしますか？あなたを漢字一文字で表すとしたら、何ですか？新たな世界を作れるとしたら、どんな世界を作りたいですか？

て何を聞きたいですか？もし動物と話せるとしたら、どの動物と何を話したいですか？もし自分が国のリーダーになったら、最初に取り組むことは何ですか？もし人生をゲームに例えるなら、今はどのステージにいると思い

ひとつの質問を抽出してください。

システム: ウェブ
本文: 「20代はがむしゃらだった」「35歳を過ぎてようやく自分の生き方を見つけた気がする...」など、「人生観が変わった!」という瞬間は人それぞれです!今回はそんな人生のターニングポイントを経験した方にインタビューを実施しました♪ぜひ参考にしてみてくださいね!!	Read more...> >> [続きを読む] «»[
--------------------------
 ウェブ
本文: 「20代はがむしゃらだった」「35歳を過ぎてようやく自分の生き方を見つけた気がする...」など、「人生観が変わった!」という瞬間は人それぞれです!今回はそんな人生のターニングポイントを経験した方にインタビューを実施しました♪ぜひ参考にしてみてくださいね!!	Read more...> >> [続きを読む] «»[
